In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/609.csv', encoding='utf-8')

In [4]:
df.head(20)

,Unnamed: 0,Gebirgsbäche im Sauerland,Plankton der Werse bei Münster,Ruhr,Lippe,Eder,Salinen und Salzgräben im südlichen Gebiet,desgl. im nördlichen Gebiet,Plankton der Talsperren,Plankton des Dortmund-Ems Kanals,"Teiche und Moor stellen ,“Kipshagen ”","Seen, Weiher und Moorstellen ,“Heiliges Meer""",Moore im Sauer und Münsterland
0,b) Rivulariaceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32.Homoeothrix spec.,+,‒,‒,‒,‒,‒,‒,‒,‒,‒,‒,‒
2,33. Calothrix (fusca?),‒,‒,‒,‒,‒,‒,‒,‒,‒,‒,+,‒
3,34. Gloeotrichia Pisum,‒,‒,‒,‒,‒,‒,‒,‒,‒,‒,+,+
4,"35. ,,intermedia",‒,‒,‒,‒,‒,‒,‒,‒,‒,‒,+,‒
5,c) Scytonemataceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,36. Tolypothrix distorta,+,‒,‒,‒,‒,‒,‒,‒,‒,‒,+,‒
7,37，，tenuis,‒,‒,‒,‒,‒,‒,‒,‒,‒,‒,+,‒
8,38. Scytonema (crispum?),‒,‒,‒,‒,‒,‒,‒,‒,‒,‒,+,‒
9,39. Desmonema Wrangelii,+,‒,‒,‒,‒,‒,‒,‒,‒,‒,‒,‒
